In [1]:
import mysql.connector
import pandas as pd

# Informations de connexion à la base de données
host = "localhost"
user = "thomas"
password = "thomas"
database = "projet"

# Connexion à la base de données
connexion = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Création d'un objet curseur pour exécuter des requêtes SQL
curseur = connexion.cursor()

In [2]:
chemin_csv = "/home/poirrier/Documents/traitement_et_donnees_reparties/Projet-NoSql/cons_maj.csv"

# Lecture du fichier CSV avec pandas
donnees_csv = pd.read_csv(chemin_csv, delimiter=";")


/tmp/ipykernel_25543/1226128616.py:4: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  donnees_csv = pd.read_csv(chemin_csv, delimiter=";")


In [3]:
donnees_csv

,annee,code_iris,nom_iris,indice_de_repetition,code_commune,nom_commune,segment_de_client,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,tri_des_adresses,code_epci,code_departement,code_region,adresse_complete,id_unique
0,2019,751145504,Petit Montrouge 4,B,75056,Paris,RESIDENTIEL,37,231.661,6.261,2.941,260001,200054781,75.0,11.0,"75 B RUE DES PLANTES, Paris",-3897590073156013273
1,2019,751145613,Plaisance 13,B,75056,Paris,RESIDENTIEL,35,36.611,1.046,2.941,260006,200054781,75.0,11.0,"24 B RUE DES PLANTES, Paris",2049997221106065414
2,2019,751145513,Petit Montrouge 13,B,75056,Paris,RESIDENTIEL,35,37.490,1.071,2.941,260007,200054781,75.0,11.0,"21 B RUE DES PLANTES, Paris",-2709076786452419388
3,2019,751145620,Plaisance 20,B,75056,Paris,RESIDENTIEL,17,53.524,3.148,2.941,260008,200054781,75.0,11.0,"16 B RUE DES PLANTES, Paris",3473625960617658766
4,2019,751207910,Pere Lachaise 10,B,75056,Paris,RESIDENTIEL,229,432.410,1.888,2.941,260010,200054781,75.0,11.0,"16 RUE DES PLATRIERES, Paris",-8182594111729820896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997643,2022,952290102,Rue de Vaux,NaN,95229,Ézanville,RESIDENTIEL,15,25.052,1.670,4.187,394059,200056380.0,95.0,11.0,"13 ALLEE DES TULIPES, Ézanville",2043583565967726668
1997644,2022,952290104,Centre Ville,NaN,95229,Ézanville,RESIDENTIEL,13,67.739,5.211,4.187,394086,200056380.0,95.0,11.0,"2 RUE SIMONE DE BEAUVOIR, Ézanville",4907811251206796333
1997645,2022,952290104,Centre Ville,NaN,95229,Ézanville,RESIDENTIEL,19,80.038,4.213,4.187,394087,200056380.0,95.0,11.0,"1 RUE SIMONE DE BEAUVOIR, Ézanville",4528127727304236957
1997646,2022,290850000,Île-Tudy (commune non irisée),B,29085,Île-Tudy,RESIDENTIEL,15,40.275,2.685,3.894,394096,242900702.0,29.0,53.0,"2 B RUE DE LA REPUBLIQUE, Île-Tudy",5142041045057789038


In [4]:
curseur.execute(f"DROP TABLE IF EXISTS Consommation;")
curseur.execute(f"TRUNCATE TABLE Adresse;")
connexion.commit()

In [5]:
curseur.execute(f"DROP TABLE IF EXISTS Adresse;")
nom_table = "Adresse"
curseur = connexion.cursor()
curseur.execute(f"""CREATE TABLE IF NOT EXISTS {nom_table}(
                id INT AUTO_INCREMENT PRIMARY KEY, 
                adresse_complete VARCHAR(255) UNIQUE, 
                commune VARCHAR(255), 
                segment_client VARCHAR(30), 
                nb_logements INT);
            """)
connexion.commit()

In [7]:
for index, row in donnees_csv.iterrows():
    # Récupérez les valeurs de chaque colonne
    adresse_complete = row['adresse_complete']
    commune = row['nom_commune']
    segment_client = row['segment_de_client']
    nb_logements = row['nombre_de_logements']
    #Utilisez une requête SQL pour insérer les données dans la table Adresse
    requete_insertion = f"""INSERT IGNORE INTO Adresse (adresse_complete, commune, segment_client, nb_logements) 
                            VALUES ("{adresse_complete}", "{commune}", "{segment_client}", {nb_logements});
                         """

    #Exécutez la requête
    curseur.execute(requete_insertion)

# N'oubliez pas de valider les changements si vous utilisez InnoDB
connexion.commit()


In [8]:
curseur.execute(f"DROP TABLE IF EXISTS Consommation;")
nom_table_consommation = "Consommation"

curseur.execute(f"""
    CREATE TABLE IF NOT EXISTS {nom_table_consommation} (
        PRIMARY KEY (id_adresse, annee),  -- Primary key
        id_adresse INT,
        consommation_mwh DOUBLE,
        annee INT,
        FOREIGN KEY (id_adresse) REFERENCES Adresse(id)
    );
""")

connexion.commit()

In [9]:
for index, row in donnees_csv.iterrows():
    # Assuming you have columns 'adresse_complete', 'commune', 'segment_client' in your CSV
    adresse_complete = row['adresse_complete']
    commune = row['nom_commune']
    segment_client = row['segment_de_client']
    consommation_mwh = row['consommation_annuelle_totale_de_l_adresse_mwh']  # Assuming you have a column 'consommation_mwh'
    annee = row['annee']  # Assuming you have a column 'annee'
  
     
    requete_insertion = f"""
    INSERT IGNORE INTO Consommation (consommation_mwh, annee, id_adresse)
    VALUES (
        {consommation_mwh},
        {annee},
        (SELECT id FROM Adresse WHERE adresse_complete = "{adresse_complete}")
        )
    """


        # Execute the SQL query
    curseur.execute(requete_insertion)

# Commit the changes to the database
connexion.commit()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Ain')
        )' at line 5

In [30]:
for index, row in donnees_csv.iterrows():
    # Assuming you have columns 'adresse_complete', 'commune', 'segment_client' in your CSV
    adresse_complete = row['adresse_complete']
    commune = row['nom_commune']
    segment_client = row['segment_de_client']

    # Use a SQL query to fetch the id_adresse from the Adresse table
    requete_select_adresse = f"""
        SELECT id FROM Adresse
        WHERE adresse_complete = '{adresse_complete}'
    """
    
    # Execute the SQL query to fetch the id_adresse
    curseur.execute(requete_select_adresse)
    result = curseur.fetchone()

    if result:
        id_adresse = result[0]
        consommation_mwh = row['consommation_annuelle_totale_de_l_adresse_mwh']  # Assuming you have a column 'consommation_mwh'
        annee = row['annee']  # Assuming you have a column 'annee'

        # Use a SQL query to insert data into the Consommation table
        requete_insertion = f"""
            INSERT INTO {nom_table_consommation} (id_adresse, consommation_mwh, annee)
            VALUES ({id_adresse}, {consommation_mwh}, {annee});
        """
        requete_insertion = f"""
            INSERT INTO Consommation
            SET consommation_mwh = {consommation_mwh},
                annee = {annee},
                id_adresse = (
                SELECT id
                    FROM Adresse
                    WHERE adresse = {adresse_complete})
        """


        # Execute the SQL query
        curseur.execute(requete_insertion)

# Commit the changes to the database
connexion.commit()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'B RUE DES PLANTES, Paris)' at line 7

In [ ]:
# Insérer les données dans la table
for index, row in donnees_csv.iterrows():
    curseur.execute(f"""
        INSERT INTO {nom_table} (adresse_complete, nom_commune, segment_de_client, nombre_de_logements)
        VALUES (%s, %s, %s, %s)
    """, (row['adresse_complete'], row['nom_commune'], row['segment_de_client'], row['nombre_de_logements']))


KeyError: 'adresse_complete'

In [ ]:
# Nom de la table dans la base de données
nom_table = "Test"


curseur.execute("DROP TABLE IF EXISTS Test")

requete_creation_table = "CREATE TABLE IF NOT EXISTS Test (annee int, code_iris int, nom_iris varchar(255), numero_de_voie int, indice_de_repetition varchar(255), type_de_voie varchar(255), libelle_de_voie varchar(255), code_commune int, nom_commune varchar(255), segment_de_client varchar(255), nombre_de_logements int, consommation_annuelle_totale_de_l_adresse_mwh float, consommation_annuelle_moyenne_par_site_de_l_adresse_mwh float, consommation_annuelle_moyenne_de_la_commune_mwh float, adresse varchar(255), tri_des_adresses int, code_epci int, code_departement int, code_region int)"


print(requete_creation_table)
# Exécution de la requête SQL
curseur.execute(requete_creation_table)
for index, ligne in donnees_csv.iterrows():
    valeurs = tuple(ligne)
    curseur.execute(f"INSERT INTO {nom_table} VALUES {valeurs};")

# Validez les changements dans la base de données
connexion.commit()

# Fermeture du curseur et de la connexion
curseur.close()
connexion.close()
